# Cactus Identification Groundwork

## The first steps of building a model with keras.


In [1]:
## Dependencies
import keras
import os
import glob
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np





Using TensorFlow backend.


How the files should be layed out in the pwd for the below code to run

In [2]:
ls

cactus-identification-earlywork.ipynb  train/  train.csv


Using pandas to format the csv file, and make a data frame to label our data, because the pictures in the train/ directory are unlabeled. Building a data frame allows us to label the data from the train.csv file instead of sorting the photos into 2 directories. Below the first 10 photos are listed, along side their labels

In [3]:
data_frame = pd.read_csv('./train.csv')
print(data_frame[:10])

                                     id  has_cactus
0  0004be2cfeaba1c0361d39e2b000257b.jpg           1
1  000c8a36845c0208e833c79c1bffedd1.jpg           1
2  000d1e9a533f62e55c289303b072733d.jpg           1
3  0011485b40695e9138e92d0b3fb55128.jpg           1
4  0014d7a11e90b62848904c1418fc8cf2.jpg           1
5  0017c3c18ddd57a2ea6f9848c79d83d2.jpg           1
6  002134abf28af54575c18741b89dd2a4.jpg           0
7  0024320f43bdd490562246435af4f90b.jpg           0
8  002930423b9840e67e5a54afd4768a1e.jpg           1
9  00351838ebf6dff6e53056e00a1e307c.jpg           1


## Building an ImageDataGenerator

There are several steps that can be manipulated below: batch size, color, and the other parametors of the ImageDataGenerator class.

A training set and validation set are created so that we can test a model.


In [4]:
columns = ["has_cactus"]

datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
dataframe=data_frame[:12000],
directory="./train",
x_col="id",
y_col=columns,
batch_size=32,
seed=6,
shuffle=True,
class_mode="other",
target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(
dataframe=data_frame[12000:],
directory="./train",
x_col="id",
y_col=columns,
batch_size=32,
seed=6,
shuffle=True,
class_mode="other",
target_size=(32,32))



Found 12000 images.
Found 5500 images.


Fields of the model can be referenced in print statements.

In [5]:
print(train_generator.color_mode)
print(train_generator.data_format)
print(train_generator.image_shape)


rgb
channels_last
(32, 32, 3)


Convolutional Network adapted from a keras example to test to see if the data had been properly imported

In [6]:

model = keras.models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=32, 
                    validation_data=valid_generator,
                    validation_steps=32,
                    epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
32/32 [==============================] - 11s 340ms/step - loss: 0.5639 - acc: 0.7383 - val_loss: 0.5044 - val_acc: 0.7656
Epoch 2/5
32/32 [==============================] - 3s 82ms/step - loss: 0.4939 - acc: 0.7354 - val_loss: 0.4339 - val_acc: 0.7314
Epoch 3/5
32/32 [==============================] - 2s 60ms/step - loss: 0.4041 - acc: 0.7822 - val_loss: 0.3193 - val_acc: 0.8984
Epoch 4/5
32/32 [==============================] - 2s 60ms/step - loss: 0.3065 - acc: 0.8721 - val_loss: 0.2486 - val_acc: 0.9043
Epoch 5/5
32/32 [==============================] - 2s 60ms/step - loss: 0.2674 - acc: 0.9014 - val_loss: 0.2010 - val_acc: 0.9277
